In [1]:
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
# DATA_DIR = config("DATA_DIR")

DATA_DIR = r"../../../FS-project_files/FX-files"

# Initial Pull and analysis

FX_MONTHLY = r"fx_monthly_data.parquet"
FX_DAILY = r"fx_daily_data.parquet"

In [4]:
fx_month = pd.read_parquet(f"{DATA_DIR}/{FX_MONTHLY}")
fx_day = pd.read_parquet(f"{DATA_DIR}/{FX_DAILY}")

In [7]:
fx_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 48 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          649 non-null    datetime64[ns]
 1   exusal        649 non-null    float64       
 2   exalus        649 non-null    float64       
 3   exbzus        361 non-null    float64       
 4   excaus        649 non-null    float64       
 5   exchus        529 non-null    float64       
 6   exdnus        649 non-null    float64       
 7   exhkus        529 non-null    float64       
 8   exinus        625 non-null    float64       
 9   exjpus        649 non-null    float64       
 10  exkous        526 non-null    float64       
 11  exmaus        649 non-null    float64       
 12  exmxus        375 non-null    float64       
 13  exusnz        649 non-null    float64       
 14  exnzus        649 non-null    float64       
 15  exnous        649 non-null    float64   

In [9]:
fx_month.columns

Index(['date', 'exusal', 'exalus', 'exbzus', 'excaus', 'exchus', 'exdnus',
       'exhkus', 'exinus', 'exjpus', 'exkous', 'exmaus', 'exmxus', 'exusnz',
       'exnzus', 'exnous', 'exsius', 'exsfus', 'exslus', 'exsdus', 'exszus',
       'extaus', 'exthus', 'exusuk', 'exukus', 'exvzus', 'exauus', 'exbeus',
       'exfnus', 'exfrus', 'exgeus', 'exgrus', 'exusir', 'exirus', 'exitus',
       'exneus', 'expous', 'exspus', 'exusec', 'execus', 'exuseu', 'exeuus',
       'twexbmth', 'twexbgsmth', 'twexmmth', 'twexafegsmth', 'twexomth',
       'twexemegsmth'],
      dtype='object')

In [10]:
fx_day.columns

Index(['date', 'dexusal', 'dexalus', 'dexbzus', 'dexcaus', 'dexchus',
       'dexdnus', 'dexhkus', 'dexinus', 'dexjpus', 'dexkous', 'dexmaus',
       'dexmxus', 'dexusnz', 'dexnzus', 'dexnous', 'dexsius', 'dexsfus',
       'dexslus', 'dexsdus', 'dexszus', 'dextaus', 'dexthus', 'dexusuk',
       'dexukus', 'dexvzus', 'exauus', 'exbeus', 'exfnus', 'exfrus', 'exgeus',
       'exgrus', 'exusir', 'exirus', 'exitus', 'exneus', 'expous', 'exspus',
       'exusec', 'execus', 'dexuseu', 'dexeuus', 'dtwexb', 'dtwexbgs',
       'dtwexm', 'dtwexafegs', 'dtwexo', 'dtwexemegs', 'indexgx'],
      dtype='object')